*Para efectos de Google Colab, primero se deben cargar todos los archivos de la carpeta MNIST por separado, luego crear las dos carpetas (processed y raw) y agregar los archivos correspondientes a cada carpeta para finalmente crear una nueva carpeta llamada MNIST donde se deben agregar las dos carpetas anteriormente mencionadas con los archivos correspondientes a cada carpeta antes de iniciar. Además se deben correr todas las casillas de código en orden para obtener los resultados correctamente.*


---


Primero importamos las librerías para el desarrollo de cada ejercicio

In [ ]:
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
import time

Luego implementamos la función entregada en main.py para calcular la precisión del conjunto de testeo

In [ ]:
# Funcion que calcula la precision sobre el conjunto de testeo  
def calculateAccuracy(model, test_loader):
      
  correct = 0
  total = 0
  # Iteramos por el conjunto de testeo
  for images, labels in test_loader:
    # Transformamos imagenes a vectores
    images = images.view(-1, 28*28).requires_grad_()

    # Pasada forward
    outputs = model(images)

    # Obtenemos la prediccion de la red: la clase con mayor valor
    _, predicted = torch.max(outputs.data, 1)

    # numero total de labels en el batch
    total += labels.size(0)

    # total predicciones correctas
    correct += (predicted == labels).sum()

  accuracy = 100 * correct / total
  return accuracy

Aquí es donde se implementa la clase solicitada. El procedimiento de como se implementa esta detallada en los comentarios del código.

In [ ]:
class FFNN_1HL(torch.nn.Module):   #creamos la clase
  def __init__(self, input_dim, hidden1_dim, output_dim): #definimos el constructor de la clase
    super(FFNN_1HL, self).__init__()
    self.fc1 = torch.nn.Linear(input_dim, hidden1_dim, bias=True) #inicializa capa de entrada y una capa oculta
    self.act1 = torch.nn.ReLU() #se asigna la funcion de activación ReLU
    self.fc2 = torch.nn.Linear(hidden1_dim, output_dim, bias=True) #inicializa la capa de salida

  def forward(self, x): #definir funcion forward
    z1 = self.fc1(x)
    h1 = self.act1(z1)
    z = self.fc2(h1) # No aplicamos softmax, ya que la funcion torch.nn.CrossEntropyLoss() la aplica
    return z

Luego aplicamos la clase creada a cada ejemplo solicitado. Los análisis entre los casos se encuentran detallados en el informe entregado.

Se entrena y prueba el modelo con descenso estocástico por gradiente simple, tasa de aprendizaje de 0.01 y dimensión de la capa oculta igual a 1.

In [ ]:
#Entrenamiento y prueba de la red con el dataset MNIST

#Código para cargar los datos

train_dataset = torchvision.datasets.MNIST(root='./', train=True, transform=transforms.ToTensor())

test_dataset = torchvision.datasets.MNIST(root='./', train=False, transform=transforms.ToTensor())

batch_size = 32  # No cambiar tamaño del batch

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

########################################
# Construimos la red
# Las dimensiones de entrada y salida corresponden al dataset MNIST

input_dim = 28*28
hidden_dim = 1
output_dim = 10

model = FFNN_1HL(input_dim, hidden_dim, output_dim)

###############################
# Definimos la funcion de perdida, en este caso entropia cruzada
# Ojo: esta funcion calcula softmax y luego la entropia cruzada vista en clases

loss_function = torch.nn.CrossEntropyLoss()

###################
#Definimos el algoritmo de optimizacion, en este caso stochastic gradient descent simple

learning_rate = 0.01

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

## Utilizar otros optimizers cuando sea necesario

####################
# Entrenamos y testeamos la red. No es necesario cambiar este código.
# Imprimimos la precision antes de comenzar, la precision y funcion de perdida despues de cada epoca,
# y el tiempo total de entrenamiento

num_epochs = 10  # No cambiar cantidad de epocas

accuracy = calculateAccuracy(model, test_loader)
print('Initial Accuracy: {}'.format(accuracy))

tiempo_epochs = 0
for epoch in range(num_epochs):
    inicio_epoch = time.process_time()
    for images, labels in train_loader:
        # Transformamos imagenes a vectores 
        images = images.view(-1, 28*28).requires_grad_()

        # Pasada forward
        outputs = model(images)

        # Calculamos la pérdida
        loss = loss_function(outputs, labels)

        # Hacemos la pasada backward
        # Esto rellena el atributo .grad de todos los parametros
        loss.backward()

        # Hacemos un paso en el algoritmo de optimizacion
        optimizer.step()

        # Inicializamos los gradientes de los parametros a 0
        optimizer.zero_grad()

    tiempo_epochs += time.process_time() - inicio_epoch
    ###############################
    # calculamos la precision sobre el conjunto de testeo        
    accuracy = calculateAccuracy(model, test_loader)
    

    # Reporte
    #print('Epoch: {}. Loss: {}'.format(epoch, loss.item()))
    print('Epoch: {}. Loss: {}. Accuracy: {}'.format(epoch, loss.item(), accuracy))

print('Total time: {}'.format(tiempo_epochs))

Initial Accuracy: 8.920000076293945
Epoch: 0. Loss: 2.04119610786438. Accuracy: 21.649999618530273
Epoch: 1. Loss: 1.8716106414794922. Accuracy: 21.510000228881836
Epoch: 2. Loss: 1.7473570108413696. Accuracy: 25.43000030517578
Epoch: 3. Loss: 1.833014726638794. Accuracy: 22.959999084472656
Epoch: 4. Loss: 1.6823080778121948. Accuracy: 29.709999084472656
Epoch: 5. Loss: 1.8993515968322754. Accuracy: 31.5
Epoch: 6. Loss: 1.7012053728103638. Accuracy: 32.2400016784668
Epoch: 7. Loss: 1.737265706062317. Accuracy: 32.2599983215332
Epoch: 8. Loss: 1.6939116716384888. Accuracy: 32.540000915527344
Epoch: 9. Loss: 1.7451506853103638. Accuracy: 30.5
Total time: 59.683837499999925


Se entrena y prueba el modelo con descenso estocástico por gradiente simple, tasa de aprendizaje de 0.01 y dimensión de la capa oculta igual a 10.

In [ ]:
#Entrenamiento y prueba de la red con el dataset MNIST

#Código para cargar los datos

train_dataset = torchvision.datasets.MNIST(root='./', train=True, transform=transforms.ToTensor())

test_dataset = torchvision.datasets.MNIST(root='./', train=False, transform=transforms.ToTensor())

batch_size = 32  # No cambiar tamaño del batch

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

########################################
# Construimos la red
# Las dimensiones de entrada y salida corresponden al dataset MNIST

input_dim = 28*28
hidden_dim = 10
output_dim = 10

model = FFNN_1HL(input_dim, hidden_dim, output_dim)

###############################
# Definimos la funcion de perdida, en este caso entropia cruzada
# Ojo: esta funcion calcula softmax y luego la entropia cruzada vista en clases

loss_function = torch.nn.CrossEntropyLoss()

###################
#Definimos el algoritmo de optimizacion, en este caso stochastic gradient descent simple

learning_rate = 0.01

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

## Utilizar otros optimizers cuando sea necesario

####################
# Entrenamos y testeamos la red. No es necesario cambiar este código.
# Imprimimos la precision antes de comenzar, la precision y funcion de perdida despues de cada epoca,
# y el tiempo total de entrenamiento

num_epochs = 10  # No cambiar cantidad de epocas

accuracy = calculateAccuracy(model, test_loader)
print('Initial Accuracy: {}'.format(accuracy))

tiempo_epochs = 0
for epoch in range(num_epochs):
    inicio_epoch = time.process_time()
    for images, labels in train_loader:
        # Transformamos imagenes a vectores 
        images = images.view(-1, 28*28).requires_grad_()

        # Pasada forward
        outputs = model(images)

        # Calculamos la pérdida
        loss = loss_function(outputs, labels)

        # Hacemos la pasada backward
        # Esto rellena el atributo .grad de todos los parametros
        loss.backward()

        # Hacemos un paso en el algoritmo de optimizacion
        optimizer.step()

        # Inicializamos los gradientes de los parametros a 0
        optimizer.zero_grad()

    tiempo_epochs += time.process_time() - inicio_epoch
    ###############################
    # calculamos la precision sobre el conjunto de testeo        
    accuracy = calculateAccuracy(model, test_loader)
    

    # Reporte
    #print('Epoch: {}. Loss: {}'.format(epoch, loss.item()))
    print('Epoch: {}. Loss: {}. Accuracy: {}'.format(epoch, loss.item(), accuracy))

print('Total time: {}'.format(tiempo_epochs))

Initial Accuracy: 9.65999984741211
Epoch: 0. Loss: 0.4459160268306732. Accuracy: 88.37999725341797
Epoch: 1. Loss: 0.28712576627731323. Accuracy: 90.18000030517578
Epoch: 2. Loss: 0.4649253189563751. Accuracy: 91.16000366210938
Epoch: 3. Loss: 0.3090496063232422. Accuracy: 91.62000274658203
Epoch: 4. Loss: 0.3345477879047394. Accuracy: 91.87000274658203
Epoch: 5. Loss: 0.2712971568107605. Accuracy: 92.13999938964844
Epoch: 6. Loss: 0.32752951979637146. Accuracy: 92.02999877929688
Epoch: 7. Loss: 0.1763847917318344. Accuracy: 92.41000366210938
Epoch: 8. Loss: 0.07958599179983139. Accuracy: 92.37999725341797
Epoch: 9. Loss: 0.1716882288455963. Accuracy: 92.55999755859375
Total time: 60.809273859


Se entrena y prueba el modelo con descenso estocástico por gradiente simple, tasa de aprendizaje de 0.01 y dimensión de la capa oculta igual a 100.

In [ ]:
#Entrenamiento y prueba de la red con el dataset MNIST

#Código para cargar los datos

train_dataset = torchvision.datasets.MNIST(root='./', train=True, transform=transforms.ToTensor())

test_dataset = torchvision.datasets.MNIST(root='./', train=False, transform=transforms.ToTensor())

batch_size = 32  # No cambiar tamaño del batch

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

########################################
# Construimos la red
# Las dimensiones de entrada y salida corresponden al dataset MNIST

input_dim = 28*28
hidden_dim = 100
output_dim = 10

model = FFNN_1HL(input_dim, hidden_dim, output_dim)

###############################
# Definimos la funcion de perdida, en este caso entropia cruzada
# Ojo: esta funcion calcula softmax y luego la entropia cruzada vista en clases

loss_function = torch.nn.CrossEntropyLoss()

###################
#Definimos el algoritmo de optimizacion, en este caso stochastic gradient descent simple

learning_rate = 0.01

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

## Utilizar otros optimizers cuando sea necesario

####################
# Entrenamos y testeamos la red. No es necesario cambiar este código.
# Imprimimos la precision antes de comenzar, la precision y funcion de perdida despues de cada epoca,
# y el tiempo total de entrenamiento

num_epochs = 10  # No cambiar cantidad de epocas

accuracy = calculateAccuracy(model, test_loader)
print('Initial Accuracy: {}'.format(accuracy))

tiempo_epochs = 0
for epoch in range(num_epochs):
    inicio_epoch = time.process_time()
    for images, labels in train_loader:
        # Transformamos imagenes a vectores 
        images = images.view(-1, 28*28).requires_grad_()

        # Pasada forward
        outputs = model(images)

        # Calculamos la pérdida
        loss = loss_function(outputs, labels)

        # Hacemos la pasada backward
        # Esto rellena el atributo .grad de todos los parametros
        loss.backward()

        # Hacemos un paso en el algoritmo de optimizacion
        optimizer.step()

        # Inicializamos los gradientes de los parametros a 0
        optimizer.zero_grad()

    tiempo_epochs += time.process_time() - inicio_epoch
    ###############################
    # calculamos la precision sobre el conjunto de testeo        
    accuracy = calculateAccuracy(model, test_loader)
    

    # Reporte
    #print('Epoch: {}. Loss: {}'.format(epoch, loss.item()))
    print('Epoch: {}. Loss: {}. Accuracy: {}'.format(epoch, loss.item(), accuracy))

print('Total time: {}'.format(tiempo_epochs))

Initial Accuracy: 12.069999694824219
Epoch: 0. Loss: 0.3579598069190979. Accuracy: 89.58000183105469
Epoch: 1. Loss: 0.5179370045661926. Accuracy: 90.81999969482422
Epoch: 2. Loss: 0.5793843865394592. Accuracy: 91.87000274658203
Epoch: 3. Loss: 0.17780552804470062. Accuracy: 92.30999755859375
Epoch: 4. Loss: 0.1683051586151123. Accuracy: 92.87000274658203
Epoch: 5. Loss: 0.05440385267138481. Accuracy: 93.33999633789062
Epoch: 6. Loss: 0.1325538009405136. Accuracy: 93.56999969482422
Epoch: 7. Loss: 0.19975878298282623. Accuracy: 93.91000366210938
Epoch: 8. Loss: 0.10683401674032211. Accuracy: 94.27999877929688
Epoch: 9. Loss: 0.09696470946073532. Accuracy: 94.38999938964844
Total time: 71.03732909000001


Se entrena y prueba el modelo con descenso estocástico por gradiente simple, tasa de aprendizaje de 0.01 y dimensión de la capa oculta igual a 200.

In [ ]:
#Entrenamiento y prueba de la red con el dataset MNIST

#Código para cargar los datos

train_dataset = torchvision.datasets.MNIST(root='./', train=True, transform=transforms.ToTensor())

test_dataset = torchvision.datasets.MNIST(root='./', train=False, transform=transforms.ToTensor())

batch_size = 32  # No cambiar tamaño del batch

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

########################################
# Construimos la red
# Las dimensiones de entrada y salida corresponden al dataset MNIST

input_dim = 28*28
hidden_dim = 200
output_dim = 10

model = FFNN_1HL(input_dim, hidden_dim, output_dim)

###############################
# Definimos la funcion de perdida, en este caso entropia cruzada
# Ojo: esta funcion calcula softmax y luego la entropia cruzada vista en clases

loss_function = torch.nn.CrossEntropyLoss()

###################
#Definimos el algoritmo de optimizacion, en este caso stochastic gradient descent simple

learning_rate = 0.01

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

## Utilizar otros optimizers cuando sea necesario

####################
# Entrenamos y testeamos la red. No es necesario cambiar este código.
# Imprimimos la precision antes de comenzar, la precision y funcion de perdida despues de cada epoca,
# y el tiempo total de entrenamiento

num_epochs = 10  # No cambiar cantidad de epocas

accuracy = calculateAccuracy(model, test_loader)
print('Initial Accuracy: {}'.format(accuracy))

tiempo_epochs = 0
for epoch in range(num_epochs):
    inicio_epoch = time.process_time()
    for images, labels in train_loader:
        # Transformamos imagenes a vectores 
        images = images.view(-1, 28*28).requires_grad_()

        # Pasada forward
        outputs = model(images)

        # Calculamos la pérdida
        loss = loss_function(outputs, labels)

        # Hacemos la pasada backward
        # Esto rellena el atributo .grad de todos los parametros
        loss.backward()

        # Hacemos un paso en el algoritmo de optimizacion
        optimizer.step()

        # Inicializamos los gradientes de los parametros a 0
        optimizer.zero_grad()

    tiempo_epochs += time.process_time() - inicio_epoch
    ###############################
    # calculamos la precision sobre el conjunto de testeo        
    accuracy = calculateAccuracy(model, test_loader)
    

    # Reporte
    #print('Epoch: {}. Loss: {}'.format(epoch, loss.item()))
    print('Epoch: {}. Loss: {}. Accuracy: {}'.format(epoch, loss.item(), accuracy))

print('Total time: {}'.format(tiempo_epochs))

Initial Accuracy: 8.1899995803833
Epoch: 0. Loss: 0.4203987419605255. Accuracy: 89.22000122070312
Epoch: 1. Loss: 0.19810040295124054. Accuracy: 90.93000030517578
Epoch: 2. Loss: 0.25252336263656616. Accuracy: 91.91999816894531
Epoch: 3. Loss: 0.5371927618980408. Accuracy: 92.47000122070312
Epoch: 4. Loss: 0.2549794018268585. Accuracy: 93.05999755859375
Epoch: 5. Loss: 0.2242819219827652. Accuracy: 93.38999938964844
Epoch: 6. Loss: 0.1658615618944168. Accuracy: 93.88999938964844
Epoch: 7. Loss: 0.11949853599071503. Accuracy: 94.02999877929688
Epoch: 8. Loss: 0.04738394543528557. Accuracy: 94.58999633789062
Epoch: 9. Loss: 0.12689921259880066. Accuracy: 94.79000091552734
Total time: 77.47349037500015


Se entrena y prueba el modelo con descenso estocástico por gradiente simple, tasa de aprendizaje de 0.01 y dimensión de la capa oculta igual a 500.

In [ ]:
#Entrenamiento y prueba de la red con el dataset MNIST

#Código para cargar los datos

train_dataset = torchvision.datasets.MNIST(root='./', train=True, transform=transforms.ToTensor())

test_dataset = torchvision.datasets.MNIST(root='./', train=False, transform=transforms.ToTensor())

batch_size = 32  # No cambiar tamaño del batch

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

########################################
# Construimos la red
# Las dimensiones de entrada y salida corresponden al dataset MNIST

input_dim = 28*28
hidden_dim = 500
output_dim = 10

model = FFNN_1HL(input_dim, hidden_dim, output_dim)

###############################
# Definimos la funcion de perdida, en este caso entropia cruzada
# Ojo: esta funcion calcula softmax y luego la entropia cruzada vista en clases

loss_function = torch.nn.CrossEntropyLoss()

###################
#Definimos el algoritmo de optimizacion, en este caso stochastic gradient descent simple

learning_rate = 0.01

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

## Utilizar otros optimizers cuando sea necesario

####################
# Entrenamos y testeamos la red. No es necesario cambiar este código.
# Imprimimos la precision antes de comenzar, la precision y funcion de perdida despues de cada epoca,
# y el tiempo total de entrenamiento

num_epochs = 10  # No cambiar cantidad de epocas

accuracy = calculateAccuracy(model, test_loader)
print('Initial Accuracy: {}'.format(accuracy))

tiempo_epochs = 0
for epoch in range(num_epochs):
    inicio_epoch = time.process_time()
    for images, labels in train_loader:
        # Transformamos imagenes a vectores 
        images = images.view(-1, 28*28).requires_grad_()

        # Pasada forward
        outputs = model(images)

        # Calculamos la pérdida
        loss = loss_function(outputs, labels)

        # Hacemos la pasada backward
        # Esto rellena el atributo .grad de todos los parametros
        loss.backward()

        # Hacemos un paso en el algoritmo de optimizacion
        optimizer.step()

        # Inicializamos los gradientes de los parametros a 0
        optimizer.zero_grad()

    tiempo_epochs += time.process_time() - inicio_epoch
    ###############################
    # calculamos la precision sobre el conjunto de testeo        
    accuracy = calculateAccuracy(model, test_loader)
    

    # Reporte
    #print('Epoch: {}. Loss: {}'.format(epoch, loss.item()))
    print('Epoch: {}. Loss: {}. Accuracy: {}'.format(epoch, loss.item(), accuracy))

print('Total time: {}'.format(tiempo_epochs))

Initial Accuracy: 9.5
Epoch: 0. Loss: 0.5301414132118225. Accuracy: 89.45999908447266
Epoch: 1. Loss: 0.31317245960235596. Accuracy: 90.94000244140625
Epoch: 2. Loss: 0.23662233352661133. Accuracy: 92.12999725341797
Epoch: 3. Loss: 0.3042011857032776. Accuracy: 92.7699966430664
Epoch: 4. Loss: 0.1294451653957367. Accuracy: 93.22000122070312
Epoch: 5. Loss: 0.1885395050048828. Accuracy: 93.7699966430664
Epoch: 6. Loss: 0.28076866269111633. Accuracy: 94.08999633789062
Epoch: 7. Loss: 0.14509224891662598. Accuracy: 94.5199966430664
Epoch: 8. Loss: 0.07743940502405167. Accuracy: 94.69999694824219
Epoch: 9. Loss: 0.16754432022571564. Accuracy: 95.05000305175781
Total time: 104.65108874200007


Se entrena y prueba el modelo con descenso estocástico por gradiente simple, tasa de aprendizaje de 0.1 y dimensión de la capa oculta igual a 500.

In [ ]:
#Entrenamiento y prueba de la red con el dataset MNIST

#Código para cargar los datos

train_dataset = torchvision.datasets.MNIST(root='./', train=True, transform=transforms.ToTensor())

test_dataset = torchvision.datasets.MNIST(root='./', train=False, transform=transforms.ToTensor())

batch_size = 32  # No cambiar tamaño del batch

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

########################################
# Construimos la red
# Las dimensiones de entrada y salida corresponden al dataset MNIST

input_dim = 28*28
hidden_dim = 500
output_dim = 10

model = FFNN_1HL(input_dim, hidden_dim, output_dim)

###############################
# Definimos la funcion de perdida, en este caso entropia cruzada
# Ojo: esta funcion calcula softmax y luego la entropia cruzada vista en clases

loss_function = torch.nn.CrossEntropyLoss()

###################
#Definimos el algoritmo de optimizacion, en este caso stochastic gradient descent simple

learning_rate = 0.1

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

## Utilizar otros optimizers cuando sea necesario

####################
# Entrenamos y testeamos la red. No es necesario cambiar este código.
# Imprimimos la precision antes de comenzar, la precision y funcion de perdida despues de cada epoca,
# y el tiempo total de entrenamiento

num_epochs = 10  # No cambiar cantidad de epocas

accuracy = calculateAccuracy(model, test_loader)
print('Initial Accuracy: {}'.format(accuracy))

tiempo_epochs = 0
for epoch in range(num_epochs):
    inicio_epoch = time.process_time()
    for images, labels in train_loader:
        # Transformamos imagenes a vectores 
        images = images.view(-1, 28*28).requires_grad_()

        # Pasada forward
        outputs = model(images)

        # Calculamos la pérdida
        loss = loss_function(outputs, labels)

        # Hacemos la pasada backward
        # Esto rellena el atributo .grad de todos los parametros
        loss.backward()

        # Hacemos un paso en el algoritmo de optimizacion
        optimizer.step()

        # Inicializamos los gradientes de los parametros a 0
        optimizer.zero_grad()

    tiempo_epochs += time.process_time() - inicio_epoch
    ###############################
    # calculamos la precision sobre el conjunto de testeo        
    accuracy = calculateAccuracy(model, test_loader)
    

    # Reporte
    #print('Epoch: {}. Loss: {}'.format(epoch, loss.item()))
    print('Epoch: {}. Loss: {}. Accuracy: {}'.format(epoch, loss.item(), accuracy))

print('Total time: {}'.format(tiempo_epochs))

Initial Accuracy: 6.539999961853027
Epoch: 0. Loss: 0.1672700047492981. Accuracy: 95.12999725341797
Epoch: 1. Loss: 0.36417534947395325. Accuracy: 96.77999877929688
Epoch: 2. Loss: 0.06076236814260483. Accuracy: 97.33000183105469
Epoch: 3. Loss: 0.06613995134830475. Accuracy: 97.20999908447266
Epoch: 4. Loss: 0.07893387228250504. Accuracy: 97.72000122070312
Epoch: 5. Loss: 0.0106982896104455. Accuracy: 97.93000030517578
Epoch: 6. Loss: 0.13810506463050842. Accuracy: 97.88999938964844
Epoch: 7. Loss: 0.01612047292292118. Accuracy: 97.83999633789062
Epoch: 8. Loss: 0.0020970746409147978. Accuracy: 97.94999694824219
Epoch: 9. Loss: 0.0035078502260148525. Accuracy: 98.16000366210938
Total time: 102.90320013399992


Se entrena y prueba el modelo con descenso estocástico por gradiente simple, tasa de aprendizaje de 0.001 y dimensión de la capa oculta igual a 500.

In [ ]:
#Entrenamiento y prueba de la red con el dataset MNIST

#Código para cargar los datos

train_dataset = torchvision.datasets.MNIST(root='./', train=True, transform=transforms.ToTensor())

test_dataset = torchvision.datasets.MNIST(root='./', train=False, transform=transforms.ToTensor())

batch_size = 32  # No cambiar tamaño del batch

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

########################################
# Construimos la red
# Las dimensiones de entrada y salida corresponden al dataset MNIST

input_dim = 28*28
hidden_dim = 500
output_dim = 10

model = FFNN_1HL(input_dim, hidden_dim, output_dim)

###############################
# Definimos la funcion de perdida, en este caso entropia cruzada
# Ojo: esta funcion calcula softmax y luego la entropia cruzada vista en clases

loss_function = torch.nn.CrossEntropyLoss()

###################
#Definimos el algoritmo de optimizacion, en este caso stochastic gradient descent simple

learning_rate = 0.001

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

## Utilizar otros optimizers cuando sea necesario

####################
# Entrenamos y testeamos la red. No es necesario cambiar este código.
# Imprimimos la precision antes de comenzar, la precision y funcion de perdida despues de cada epoca,
# y el tiempo total de entrenamiento

num_epochs = 10  # No cambiar cantidad de epocas

accuracy = calculateAccuracy(model, test_loader)
print('Initial Accuracy: {}'.format(accuracy))

tiempo_epochs = 0
for epoch in range(num_epochs):
    inicio_epoch = time.process_time()
    for images, labels in train_loader:
        # Transformamos imagenes a vectores 
        images = images.view(-1, 28*28).requires_grad_()

        # Pasada forward
        outputs = model(images)

        # Calculamos la pérdida
        loss = loss_function(outputs, labels)

        # Hacemos la pasada backward
        # Esto rellena el atributo .grad de todos los parametros
        loss.backward()

        # Hacemos un paso en el algoritmo de optimizacion
        optimizer.step()

        # Inicializamos los gradientes de los parametros a 0
        optimizer.zero_grad()

    tiempo_epochs += time.process_time() - inicio_epoch
    ###############################
    # calculamos la precision sobre el conjunto de testeo        
    accuracy = calculateAccuracy(model, test_loader)
    

    # Reporte
    #print('Epoch: {}. Loss: {}'.format(epoch, loss.item()))
    print('Epoch: {}. Loss: {}. Accuracy: {}'.format(epoch, loss.item(), accuracy))

print('Total time: {}'.format(tiempo_epochs))

Initial Accuracy: 17.40999984741211
Epoch: 0. Loss: 1.798116683959961. Accuracy: 74.5
Epoch: 1. Loss: 1.1157560348510742. Accuracy: 80.97000122070312
Epoch: 2. Loss: 0.8444420099258423. Accuracy: 83.91999816894531
Epoch: 3. Loss: 0.7269243597984314. Accuracy: 86.02999877929688
Epoch: 4. Loss: 0.44353482127189636. Accuracy: 87.0
Epoch: 5. Loss: 0.3937283754348755. Accuracy: 87.7699966430664
Epoch: 6. Loss: 0.4958999752998352. Accuracy: 88.51000213623047
Epoch: 7. Loss: 0.5088504552841187. Accuracy: 89.02999877929688
Epoch: 8. Loss: 0.41216573119163513. Accuracy: 89.33999633789062
Epoch: 9. Loss: 0.6187611222267151. Accuracy: 89.66000366210938
Total time: 103.15777044599974


Se entrena y prueba el modelo con descenso estocástico por gradiente simple, tasa de aprendizaje de 0.0001 y dimensión de la capa oculta igual a 500.

In [ ]:
#Entrenamiento y prueba de la red con el dataset MNIST

#Código para cargar los datos

train_dataset = torchvision.datasets.MNIST(root='./', train=True, transform=transforms.ToTensor())

test_dataset = torchvision.datasets.MNIST(root='./', train=False, transform=transforms.ToTensor())

batch_size = 32  # No cambiar tamaño del batch

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

########################################
# Construimos la red
# Las dimensiones de entrada y salida corresponden al dataset MNIST

input_dim = 28*28
hidden_dim = 500
output_dim = 10

model = FFNN_1HL(input_dim, hidden_dim, output_dim)

###############################
# Definimos la funcion de perdida, en este caso entropia cruzada
# Ojo: esta funcion calcula softmax y luego la entropia cruzada vista en clases

loss_function = torch.nn.CrossEntropyLoss()

###################
#Definimos el algoritmo de optimizacion, en este caso stochastic gradient descent simple

learning_rate = 0.0001

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

## Utilizar otros optimizers cuando sea necesario

####################
# Entrenamos y testeamos la red. No es necesario cambiar este código.
# Imprimimos la precision antes de comenzar, la precision y funcion de perdida despues de cada epoca,
# y el tiempo total de entrenamiento

num_epochs = 10  # No cambiar cantidad de epocas

accuracy = calculateAccuracy(model, test_loader)
print('Initial Accuracy: {}'.format(accuracy))

tiempo_epochs = 0
for epoch in range(num_epochs):
    inicio_epoch = time.process_time()
    for images, labels in train_loader:
        # Transformamos imagenes a vectores 
        images = images.view(-1, 28*28).requires_grad_()

        # Pasada forward
        outputs = model(images)

        # Calculamos la pérdida
        loss = loss_function(outputs, labels)

        # Hacemos la pasada backward
        # Esto rellena el atributo .grad de todos los parametros
        loss.backward()

        # Hacemos un paso en el algoritmo de optimizacion
        optimizer.step()

        # Inicializamos los gradientes de los parametros a 0
        optimizer.zero_grad()

    tiempo_epochs += time.process_time() - inicio_epoch
    ###############################
    # calculamos la precision sobre el conjunto de testeo        
    accuracy = calculateAccuracy(model, test_loader)
    

    # Reporte
    #print('Epoch: {}. Loss: {}'.format(epoch, loss.item()))
    print('Epoch: {}. Loss: {}. Accuracy: {}'.format(epoch, loss.item(), accuracy))

print('Total time: {}'.format(tiempo_epochs))

Initial Accuracy: 8.470000267028809
Epoch: 0. Loss: 2.2619192600250244. Accuracy: 23.950000762939453
Epoch: 1. Loss: 2.22544002532959. Accuracy: 39.119998931884766
Epoch: 2. Loss: 2.1842217445373535. Accuracy: 50.9900016784668
Epoch: 3. Loss: 2.086149215698242. Accuracy: 60.290000915527344
Epoch: 4. Loss: 2.1034886837005615. Accuracy: 65.33999633789062
Epoch: 5. Loss: 2.077972650527954. Accuracy: 68.33000183105469
Epoch: 6. Loss: 1.9368476867675781. Accuracy: 70.26000213623047
Epoch: 7. Loss: 1.836832880973816. Accuracy: 71.48999786376953
Epoch: 8. Loss: 1.7858805656433105. Accuracy: 72.69000244140625
Epoch: 9. Loss: 1.8039236068725586. Accuracy: 73.66000366210938
Total time: 109.02275252200025


Se cambia la clase creada anteriormente y se cambia la funcion de activación a la función Sigmoid.

In [ ]:
# Aca debe implementar la clase FFNN_1HL para una red Feed-Forward, con una capa escondida
class FFNN_1HL(torch.nn.Module):   
  def __init__(self, input_dim, hidden1_dim, output_dim): #definir el constructor
    super(FFNN_1HL, self).__init__()
    self.fc1 = torch.nn.Linear(input_dim, hidden1_dim, bias=True) #inicializa capa de entrada y la capa oculta
    self.act1 = torch.nn.Sigmoid()
    self.fc2 = torch.nn.Linear(hidden1_dim, output_dim, bias=True)

  def forward(self, x): #definir funcion forward
        
    z1 = self.fc1(x)
    h1 = self.act1(z1)
    z = self.fc2(h1) # No aplicamos softmax, ya que la funcion torch.nn.CrossEntropyLoss() la aplica
      
    return z

Se entrena y prueba el modelo con descenso estocástico por gradiente simple, función de activación Sigmoid, tasa de aprendizaje de 0.01 y dimensión de la capa oculta igual a 500.

In [ ]:
#Entrenamiento y prueba de la red con el dataset MNIST

#Código para cargar los datos

train_dataset = torchvision.datasets.MNIST(root='./', train=True, transform=transforms.ToTensor())

test_dataset = torchvision.datasets.MNIST(root='./', train=False, transform=transforms.ToTensor())

batch_size = 32  # No cambiar tamaño del batch

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

########################################
# Construimos la red
# Las dimensiones de entrada y salida corresponden al dataset MNIST

input_dim = 28*28
hidden_dim = 500
output_dim = 10

model = FFNN_1HL(input_dim, hidden_dim, output_dim)

###############################
# Definimos la funcion de perdida, en este caso entropia cruzada
# Ojo: esta funcion calcula softmax y luego la entropia cruzada vista en clases

loss_function = torch.nn.CrossEntropyLoss()

###################
#Definimos el algoritmo de optimizacion, en este caso stochastic gradient descent simple

learning_rate = 0.01

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

## Utilizar otros optimizers cuando sea necesario

####################
# Entrenamos y testeamos la red. No es necesario cambiar este código.
# Imprimimos la precision antes de comenzar, la precision y funcion de perdida despues de cada epoca,
# y el tiempo total de entrenamiento

num_epochs = 10  # No cambiar cantidad de epocas

accuracy = calculateAccuracy(model, test_loader)
print('Initial Accuracy: {}'.format(accuracy))

tiempo_epochs = 0
for epoch in range(num_epochs):
    inicio_epoch = time.process_time()
    for images, labels in train_loader:
        # Transformamos imagenes a vectores 
        images = images.view(-1, 28*28).requires_grad_()

        # Pasada forward
        outputs = model(images)

        # Calculamos la pérdida
        loss = loss_function(outputs, labels)

        # Hacemos la pasada backward
        # Esto rellena el atributo .grad de todos los parametros
        loss.backward()

        # Hacemos un paso en el algoritmo de optimizacion
        optimizer.step()

        # Inicializamos los gradientes de los parametros a 0
        optimizer.zero_grad()

    tiempo_epochs += time.process_time() - inicio_epoch
    ###############################
    # calculamos la precision sobre el conjunto de testeo        
    accuracy = calculateAccuracy(model, test_loader)
    

    # Reporte
    #print('Epoch: {}. Loss: {}'.format(epoch, loss.item()))
    print('Epoch: {}. Loss: {}. Accuracy: {}'.format(epoch, loss.item(), accuracy))

print('Total time: {}'.format(tiempo_epochs))

Initial Accuracy: 10.279999732971191
Epoch: 0. Loss: 0.9872061610221863. Accuracy: 75.31999969482422
Epoch: 1. Loss: 0.6582845449447632. Accuracy: 85.47000122070312
Epoch: 2. Loss: 0.33700230717658997. Accuracy: 87.66999816894531
Epoch: 3. Loss: 0.4269200563430786. Accuracy: 88.79000091552734
Epoch: 4. Loss: 0.2938231825828552. Accuracy: 89.5
Epoch: 5. Loss: 0.4196559488773346. Accuracy: 89.80999755859375
Epoch: 6. Loss: 0.43551790714263916. Accuracy: 90.16000366210938
Epoch: 7. Loss: 0.32238149642944336. Accuracy: 90.41000366210938
Epoch: 8. Loss: 0.5195304751396179. Accuracy: 90.62000274658203
Epoch: 9. Loss: 0.33514270186424255. Accuracy: 90.94000244140625
Total time: 105.62819426600004


Se cambia la clase creada anteriormente y se cambia la funcion de activación a la función Tanh.

In [ ]:
# Aca debe implementar la clase FFNN_1HL para una red Feed-Forward, con una capa escondida
class FFNN_1HL(torch.nn.Module):   
  def __init__(self, input_dim, hidden1_dim, output_dim): #definir el constructor
    super(FFNN_1HL, self).__init__()
    self.fc1 = torch.nn.Linear(input_dim, hidden1_dim, bias=True) #inicializa capa de entrada y la capa oculta
    self.act1 = torch.nn.Tanh()
    self.fc2 = torch.nn.Linear(hidden1_dim, output_dim, bias=True)

  def forward(self, x): #definir funcion forward
        
    z1 = self.fc1(x)
    h1 = self.act1(z1)
    z = self.fc2(h1) # No aplicamos softmax, ya que la funcion torch.nn.CrossEntropyLoss() la aplica
      
    return z

Se entrena y prueba el modelo con descenso estocástico por gradiente simple, función de activación Tanh, tasa de aprendizaje de 0.01 y dimensión de la capa oculta igual a 500.

In [ ]:
#Entrenamiento y prueba de la red con el dataset MNIST

#Código para cargar los datos

train_dataset = torchvision.datasets.MNIST(root='./', train=True, transform=transforms.ToTensor())

test_dataset = torchvision.datasets.MNIST(root='./', train=False, transform=transforms.ToTensor())

batch_size = 32  # No cambiar tamaño del batch

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

########################################
# Construimos la red
# Las dimensiones de entrada y salida corresponden al dataset MNIST

input_dim = 28*28
hidden_dim = 500
output_dim = 10

model = FFNN_1HL(input_dim, hidden_dim, output_dim)

###############################
# Definimos la funcion de perdida, en este caso entropia cruzada
# Ojo: esta funcion calcula softmax y luego la entropia cruzada vista en clases

loss_function = torch.nn.CrossEntropyLoss()

###################
#Definimos el algoritmo de optimizacion, en este caso stochastic gradient descent simple

learning_rate = 0.01

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

## Utilizar otros optimizers cuando sea necesario

####################
# Entrenamos y testeamos la red. No es necesario cambiar este código.
# Imprimimos la precision antes de comenzar, la precision y funcion de perdida despues de cada epoca,
# y el tiempo total de entrenamiento

num_epochs = 10  # No cambiar cantidad de epocas

accuracy = calculateAccuracy(model, test_loader)
print('Initial Accuracy: {}'.format(accuracy))

tiempo_epochs = 0
for epoch in range(num_epochs):
    inicio_epoch = time.process_time()
    for images, labels in train_loader:
        # Transformamos imagenes a vectores 
        images = images.view(-1, 28*28).requires_grad_()

        # Pasada forward
        outputs = model(images)

        # Calculamos la pérdida
        loss = loss_function(outputs, labels)

        # Hacemos la pasada backward
        # Esto rellena el atributo .grad de todos los parametros
        loss.backward()

        # Hacemos un paso en el algoritmo de optimizacion
        optimizer.step()

        # Inicializamos los gradientes de los parametros a 0
        optimizer.zero_grad()

    tiempo_epochs += time.process_time() - inicio_epoch
    ###############################
    # calculamos la precision sobre el conjunto de testeo        
    accuracy = calculateAccuracy(model, test_loader)
    

    # Reporte
    #print('Epoch: {}. Loss: {}'.format(epoch, loss.item()))
    print('Epoch: {}. Loss: {}. Accuracy: {}'.format(epoch, loss.item(), accuracy))

print('Total time: {}'.format(tiempo_epochs))

Initial Accuracy: 13.739999771118164
Epoch: 0. Loss: 0.30576440691947937. Accuracy: 89.33000183105469
Epoch: 1. Loss: 0.2785475254058838. Accuracy: 90.52999877929688
Epoch: 2. Loss: 0.4172811806201935. Accuracy: 91.47000122070312
Epoch: 3. Loss: 0.2340552657842636. Accuracy: 91.77999877929688
Epoch: 4. Loss: 0.8662914037704468. Accuracy: 91.86000061035156
Epoch: 5. Loss: 0.12896671891212463. Accuracy: 92.18000030517578
Epoch: 6. Loss: 0.09679830819368362. Accuracy: 92.37000274658203
Epoch: 7. Loss: 0.4532102346420288. Accuracy: 92.55000305175781
Epoch: 8. Loss: 0.29645246267318726. Accuracy: 92.63999938964844
Epoch: 9. Loss: 0.34726378321647644. Accuracy: 92.87999725341797
Total time: 104.16227562999984


Se vuelve a modificar la clase usando la funcion de activación ReLU para las últimas pruebas.

In [ ]:
# Aca debe implementar la clase FFNN_1HL para una red Feed-Forward, con una capa escondida
class FFNN_1HL(torch.nn.Module):   
  def __init__(self, input_dim, hidden1_dim, output_dim): #definir el constructor
    super(FFNN_1HL, self).__init__()
    self.fc1 = torch.nn.Linear(input_dim, hidden1_dim, bias=True) #inicializa capa de entrada y la capa oculta
    self.act1 = torch.nn.ReLU()
    self.fc2 = torch.nn.Linear(hidden1_dim, output_dim, bias=True)

  def forward(self, x): #definir funcion forward
        
    z1 = self.fc1(x)
    h1 = self.act1(z1)
    z = self.fc2(h1) # No aplicamos softmax, ya que la funcion torch.nn.CrossEntropyLoss() la aplica
      
    return z

Se entrena y prueba el modelo con descenso estocástico por gradiente con momentum de 0.9, función de activación ReLU, tasa de aprendizaje de 0.01 y dimensión de la capa oculta igual a 500.

In [ ]:
#Entrenamiento y prueba de la red con el dataset MNIST

#Código para cargar los datos

train_dataset = torchvision.datasets.MNIST(root='./', train=True, transform=transforms.ToTensor())

test_dataset = torchvision.datasets.MNIST(root='./', train=False, transform=transforms.ToTensor())

batch_size = 32  # No cambiar tamaño del batch

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

########################################
# Construimos la red
# Las dimensiones de entrada y salida corresponden al dataset MNIST

input_dim = 28*28
hidden_dim = 500
output_dim = 10

model = FFNN_1HL(input_dim, hidden_dim, output_dim)

###############################
# Definimos la funcion de perdida, en este caso entropia cruzada
# Ojo: esta funcion calcula softmax y luego la entropia cruzada vista en clases

loss_function = torch.nn.CrossEntropyLoss()

###################
#Definimos el algoritmo de optimizacion, en este caso stochastic gradient descent simple

learning_rate = 0.01

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)

## Utilizar otros optimizers cuando sea necesario

####################
# Entrenamos y testeamos la red. No es necesario cambiar este código.
# Imprimimos la precision antes de comenzar, la precision y funcion de perdida despues de cada epoca,
# y el tiempo total de entrenamiento

num_epochs = 10  # No cambiar cantidad de epocas

accuracy = calculateAccuracy(model, test_loader)
print('Initial Accuracy: {}'.format(accuracy))

tiempo_epochs = 0
for epoch in range(num_epochs):
    inicio_epoch = time.process_time()
    for images, labels in train_loader:
        # Transformamos imagenes a vectores 
        images = images.view(-1, 28*28).requires_grad_()

        # Pasada forward
        outputs = model(images)

        # Calculamos la pérdida
        loss = loss_function(outputs, labels)

        # Hacemos la pasada backward
        # Esto rellena el atributo .grad de todos los parametros
        loss.backward()

        # Hacemos un paso en el algoritmo de optimizacion
        optimizer.step()

        # Inicializamos los gradientes de los parametros a 0
        optimizer.zero_grad()

    tiempo_epochs += time.process_time() - inicio_epoch
    ###############################
    # calculamos la precision sobre el conjunto de testeo        
    accuracy = calculateAccuracy(model, test_loader)
    

    # Reporte
    #print('Epoch: {}. Loss: {}'.format(epoch, loss.item()))
    print('Epoch: {}. Loss: {}. Accuracy: {}'.format(epoch, loss.item(), accuracy))

print('Total time: {}'.format(tiempo_epochs))

Initial Accuracy: 15.069999694824219
Epoch: 0. Loss: 0.20808057487010956. Accuracy: 94.69000244140625
Epoch: 1. Loss: 0.04528999328613281. Accuracy: 96.68000030517578
Epoch: 2. Loss: 0.0725172758102417. Accuracy: 97.0199966430664
Epoch: 3. Loss: 0.09029022604227066. Accuracy: 97.58000183105469
Epoch: 4. Loss: 0.007448678836226463. Accuracy: 97.58999633789062
Epoch: 5. Loss: 0.037819668650627136. Accuracy: 97.94999694824219
Epoch: 6. Loss: 0.01138770766556263. Accuracy: 97.81999969482422
Epoch: 7. Loss: 0.005334752611815929. Accuracy: 98.02999877929688
Epoch: 8. Loss: 0.05759888142347336. Accuracy: 97.75
Epoch: 9. Loss: 0.011603009887039661. Accuracy: 98.08999633789062
Total time: 143.82335730600016


Se entrena y prueba el modelo con optimizador Adam con parámetros por defecto de pytorch, función de activación ReLU, tasa de aprendizaje de 0.01 y dimensión de la capa oculta igual a 500.

In [ ]:
#Entrenamiento y prueba de la red con el dataset MNIST

#Código para cargar los datos

train_dataset = torchvision.datasets.MNIST(root='./', train=True, transform=transforms.ToTensor())

test_dataset = torchvision.datasets.MNIST(root='./', train=False, transform=transforms.ToTensor())

batch_size = 32  # No cambiar tamaño del batch

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

########################################
# Construimos la red
# Las dimensiones de entrada y salida corresponden al dataset MNIST

input_dim = 28*28
hidden_dim = 500
output_dim = 10

model = FFNN_1HL(input_dim, hidden_dim, output_dim)

###############################
# Definimos la funcion de perdida, en este caso entropia cruzada
# Ojo: esta funcion calcula softmax y luego la entropia cruzada vista en clases

loss_function = torch.nn.CrossEntropyLoss()

###################
#Definimos el algoritmo de optimizacion, en este caso stochastic gradient descent simple

learning_rate = 0.01

optimizer = torch.optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)

## Utilizar otros optimizers cuando sea necesario

####################
# Entrenamos y testeamos la red. No es necesario cambiar este código.
# Imprimimos la precision antes de comenzar, la precision y funcion de perdida despues de cada epoca,
# y el tiempo total de entrenamiento

num_epochs = 10  # No cambiar cantidad de epocas

accuracy = calculateAccuracy(model, test_loader)
print('Initial Accuracy: {}'.format(accuracy))

tiempo_epochs = 0
for epoch in range(num_epochs):
    inicio_epoch = time.process_time()
    for images, labels in train_loader:
        # Transformamos imagenes a vectores 
        images = images.view(-1, 28*28).requires_grad_()

        # Pasada forward
        outputs = model(images)

        # Calculamos la pérdida
        loss = loss_function(outputs, labels)

        # Hacemos la pasada backward
        # Esto rellena el atributo .grad de todos los parametros
        loss.backward()

        # Hacemos un paso en el algoritmo de optimizacion
        optimizer.step()

        # Inicializamos los gradientes de los parametros a 0
        optimizer.zero_grad()

    tiempo_epochs += time.process_time() - inicio_epoch
    ###############################
    # calculamos la precision sobre el conjunto de testeo        
    accuracy = calculateAccuracy(model, test_loader)
    

    # Reporte
    #print('Epoch: {}. Loss: {}'.format(epoch, loss.item()))
    print('Epoch: {}. Loss: {}. Accuracy: {}'.format(epoch, loss.item(), accuracy))

print('Total time: {}'.format(tiempo_epochs))

Initial Accuracy: 12.020000457763672
Epoch: 0. Loss: 0.11498289555311203. Accuracy: 97.0999984741211
Epoch: 1. Loss: 0.04495486244559288. Accuracy: 97.63999938964844
Epoch: 2. Loss: 0.07702187448740005. Accuracy: 97.55999755859375
Epoch: 3. Loss: 0.00246690702624619. Accuracy: 97.66000366210938
Epoch: 4. Loss: 0.07088015973567963. Accuracy: 97.98999786376953
Epoch: 5. Loss: 0.0019395618000999093. Accuracy: 98.06999969482422
Epoch: 6. Loss: 0.0010765620972961187. Accuracy: 97.69999694824219
Epoch: 7. Loss: 0.038723934441804886. Accuracy: 98.0199966430664
Epoch: 8. Loss: 0.033348117023706436. Accuracy: 98.19000244140625
Epoch: 9. Loss: 0.04845401272177696. Accuracy: 97.91000366210938
Total time: 215.92573409700026


En este código se muestran todas las salidas de cada ejecución. El análisis de los datos del tiempo de ejecución, función de pérdida y la precisión de la red están descritos en el informe entregado en formato pdf.